In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!git clone https://github.com/raghav-thiruv/Omdena-Quantum-Self-Driving.git

Cloning into 'Omdena-Quantum-Self-Driving'...
remote: Enumerating objects: 45889, done.
remote: Counting objects: 100% (11799/11799), done.
remote: Compressing objects: 100% (11746/11746), done.
remote: Total 45889 (delta 94), reused 11715 (delta 43), pack-reused 34090
Receiving objects: 100% (45889/45889), 2.16 GiB | 36.71 MiB/s, done.
Resolving deltas: 100% (123/123), done.
Updating files: 100% (45596/45596), done.


In [3]:
import tensorflow as tf

import pandas as pd
import numpy as np
import os
import keras
import random
import math
import seaborn as sns

from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

from tensorflow.keras.layers import Dense,GlobalAveragePooling2D,Convolution2D,BatchNormalization
from tensorflow.keras.layers import Flatten,MaxPooling2D,Dropout

from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications.densenet import preprocess_input

from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,img_to_array

from tensorflow.keras.models import Model

from tensorflow.keras.optimizers import Adam

from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

from keras import layers

from keras import models

from keras.layers import (Input, Dense, Activation, ZeroPadding2D,
BatchNormalization, Flatten, Conv2D, concatenate, Lambda)

from keras.layers import (AveragePooling2D, MaxPooling2D, Dropout,
GlobalMaxPooling2D, GlobalAveragePooling2D)

from keras.models import Model, load_model
from keras import regularizers, optimizers

from sklearn.model_selection import train_test_split

from keras.utils import to_categorical


import warnings
warnings.filterwarnings("ignore")

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [4]:
from struct import unpack
from tqdm import tqdm
import os

img_dir = '/content/Omdena-Quantum-Self-Driving/Images/driving_dataset1/'
root_img = '/content/Omdena-Quantum-Self-Driving/Images/driving_dataset1/'
marker_mapping = {
    0xffd8: "Start of Image",
    0xffe0: "Application Default Header",
    0xffdb: "Quantization Table",
    0xffc0: "Start of Frame",
    0xffc4: "Define Huffman Table",
    0xffda: "Start of Scan",
    0xffd9: "End of Image"
}


class JPEG:
    def __init__(self, image_file):
        with open(image_file, 'rb') as f:
            self.img_data = f.read()

    def decode(self):
        data = self.img_data
        while(True):
            marker, = unpack(">H", data[0:2])
            # print(marker_mapping.get(marker))
            if marker == 0xffd8:
                data = data[2:]
            elif marker == 0xffd9:
                return
            elif marker == 0xffda:
                data = data[-2:]
            else:
                lenchunk, = unpack(">H", data[2:4])
                data = data[2+lenchunk:]
            if len(data)==0:
               raise TypeError("issue reading jpeg file")


bads = []

for dirName, subdirList, fileList in os.walk(img_dir):
    imagesList = fileList
    for img in tqdm(imagesList):
      image = os.path.join(root_img,img)
      image = JPEG(image)
      try:
        image.decode()
      except:
        bads.append(img)


for name in bads:
  os.remove(os.path.join(root_img,name))

In [5]:
!ls

Omdena-Quantum-Self-Driving  __notebook_source__.ipynb


In [6]:
!mv "Omdena-Quantum-Self-Driving/Images/driving_dataset1/data.txt" "Omdena-Quantum-Self-Driving/Images/"

In [7]:
# Load steering angles from text file
df = pd.read_csv('Omdena-Quantum-Self-Driving/Images/data.txt', names=['filename', 'steering_angle'], delimiter=' ')
image_dir = 'Omdena-Quantum-Self-Driving/Images/driving_dataset1/'
data_file = "Omdena-Quantum-Self-Driving/Images/data.txt"
df['filename'] = df['filename'].apply(lambda x: os.path.join(image_dir, x))
df.head()

,filename,steering_angle
0,Omdena-Quantum-Self-Driving/Images/driving_dat...,0.0
1,Omdena-Quantum-Self-Driving/Images/driving_dat...,0.0
2,Omdena-Quantum-Self-Driving/Images/driving_dat...,0.0
3,Omdena-Quantum-Self-Driving/Images/driving_dat...,0.0
4,Omdena-Quantum-Self-Driving/Images/driving_dat...,0.0


In [8]:
def load_image(image_path):
    try:
        image = tf.io.read_file(image_path)
        image = tf.image.decode_jpeg(image, channels=3)
        image = tf.image.resize(image, (224, 224))
        image = tf.keras.applications.resnet.preprocess_input(image)
    except:
        print(f"Invalid image format, skipping: {image_path}")
        return None
    return image

def create_dataset(df):
    image_dataset = tf.data.Dataset.from_tensor_slices(df['filename'])
    angle_dataset = tf.data.Dataset.from_tensor_slices(df['steering_angle'])
    image_dataset = image_dataset.map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
    image_dataset = image_dataset.apply(tf.data.experimental.ignore_errors())
    dataset = tf.data.Dataset.zip((image_dataset, angle_dataset))

    return dataset

In [9]:
def densenet_model(input_shape):
    
    x_input = Input(shape=input_shape)
    x = Lambda(lambda x: x/127.5-1.0)(x_input)
    
    x = Conv2D(32,(3,3),activation='relu',padding='same')(x_input)
    
    x = Conv2D(32,(3,3),activation='relu',padding='same')(x)
    x = MaxPooling2D((2,2),padding='valid')(x)
    
    x = Conv2D(64,(3,3),activation='relu',padding='same')(x)
    
    x = Conv2D(64,(3,3),activation='relu',padding='same')(x)
    x = MaxPooling2D((2,2),padding='valid')(x)
    
    x = Conv2D(128,(3,3),activation='relu',padding='same')(x)
    x = MaxPooling2D((2,2),padding='valid')(x)
    
    x = Conv2D(128,(3,3),activation='relu',padding='same')(x)
    x = MaxPooling2D((2,2),padding='valid')(x)
    
    x = Flatten()(x)
    x = Dropout(0.5)(x)
       
    x = BatchNormalization()(x)
    x = Dense(512)(x)
    x = Dense(256)(x)
    x = Dense(64)(x)
    x = Dense(1)(x)
    
    model = Model(inputs=x_input,outputs=x,name='model')
    return model

model = densenet_model((224, 224, 3))

In [10]:
validation_split = 0.1
df = df.sample(frac=1).reset_index(drop=True)
val_df = df[:int(validation_split*len(df))]
train_df = df[int(validation_split*len(df)):]

train_dataset = create_dataset(train_df)
val_dataset = create_dataset(val_df)

batch_size = 32
train_dataset = train_dataset.shuffle(buffer_size=1000).batch(batch_size)
val_dataset = val_dataset.batch(batch_size)

In [11]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 224, 224, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 224, 224, 32)      9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 32)     0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 112, 112, 64)      18496     
                                                                 
 conv2d_3 (Conv2D)           (None, 112, 112, 64)      36928     
                                                             

In [12]:

# Define early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

# Define learning rate scheduler
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2)

# Fit the model
model.fit(train_dataset, validation_data=val_dataset, epochs=50, callbacks=[early_stopping, lr_scheduler])

Epoch 1/50
1278/1278 [==============================] - 200s 142ms/step - loss: 2.0452e-07 - accuracy: 0.0251 - val_loss: 2.3957e-07 - val_accuracy: 0.0247 - lr: 0.0010
Epoch 2/50
1278/1278 [==============================] - 182s 140ms/step - loss: 2.0452e-07 - accuracy: 0.0245 - val_loss: 2.3957e-07 - val_accuracy: 0.0249 - lr: 0.0010
Epoch 3/50
1278/1278 [==============================] - 182s 140ms/step - loss: 2.0452e-07 - accuracy: 0.0251 - val_loss: 2.3957e-07 - val_accuracy: 0.0253 - lr: 0.0010
Epoch 4/50
1278/1278 [==============================] - 194s 149ms/step - loss: 2.0452e-07 - accuracy: 0.0249 - val_loss: 2.3957e-07 - val_accuracy: 0.0238 - lr: 1.0000e-04


### Testing for a random image

In [13]:
import random
from tensorflow.keras.preprocessing import image as keras_image
import numpy as np

def load_image(image_path, target_size=(224, 224)):
    img = keras_image.load_img(image_path, target_size=target_size)
    img_tensor = keras_image.img_to_array(img)
    img_tensor = np.expand_dims(img_tensor, axis=0)
    img_tensor = tf.keras.applications.resnet.preprocess_input(img_tensor)
    return img_tensor

# Randomly select an image from validation set
random_image_path = random.choice(val_df['filename'].tolist())

# Load the image
test_image = load_image(random_image_path)

# Use the model to predict the steering angle for the test image
predicted_angle = model.predict(test_image)

# Print out the predicted steering angle
print("Predicted steering angle: ", predicted_angle[0][0])

# If you want to compare this prediction to the actual angle, you could find that as follows:
actual_angle = val_df[val_df['filename'] == random_image_path]['steering_angle'].values[0]
print("Actual steering angle: ", actual_angle)

1/1 [==============================] - 0s 283ms/step
Predicted steering angle:  1.2313507
Actual steering angle:  1.11


fatal: not a git repository (or any parent up to mount point /kaggle)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).
